## Notebook setup

In [ ]:
import os
import io
import warnings
import datetime
import numpy as np
import base64

from IPython.display import HTML

import openbb_terminal.config_terminal as cfg
from openbb_terminal.helper_classes import TerminalStyle
from openbb_terminal import OpenBBFigure, theme

warnings.filterwarnings("ignore")

In [ ]:
# Suppressing sdk logs
from openbb_terminal.core.session.current_system import set_system_variable

set_system_variable("LOGGING_SUPPRESS", True)

# Import the OpenBB SDK
from openbb_terminal.sdk import openbb, widgets

In [ ]:
from openbb_terminal.core.session.current_user import get_current_user

user = get_current_user()

cfg.theme = TerminalStyle(
    user.preferences.MPL_STYLE,
    user.preferences.PMF_STYLE,
    user.preferences.RICH_STYLE,
)
theme.apply_style("light")

In [ ]:
stylesheet = widgets.html_report_stylesheet()

In [ ]:
def get_html(func, title, *args, legend=False, **kwargs):
    kwargs["external_axes"] = True
    local_fig = func(*args, **kwargs)
    # kwargs["chart"] = True
    if title:
        local_fig.set_title(title)
        local_fig.set_xaxis_title("Lag")
        local_fig.set_yaxis_title("Autocorrelation")
    if legend:
        local_fig.update_layout(
            legend=dict(
                x=0.01,
                y=0.99,
                xanchor="left",
                yanchor="top",
                xref="paper",
                yref="paper",
            )
        )
    return local_fig.to_html()

## Select Ticker

In [ ]:
# Parameters that will be replaced when calling this notebook
# Do not leave parameters blank as notebook will not run otherwise
symbol = "VTI"
report_name = "Forecast Report for VTI"

In [ ]:
ticker = symbol

In [ ]:
if "." in ticker:
    import sys

    sys.exit(0)

In [ ]:
ticker_data = openbb.stocks.load(
    symbol=ticker, start_date=datetime.datetime.now() - datetime.timedelta(days=400)
)

author = "OpenBB"
report_title = f"FORECAST RESEARCH REPORT ON {ticker.upper()}"
report_date = datetime.datetime.now().strftime("%d %B, %Y")
report_time = datetime.datetime.now().strftime("%H:%M")
report_timezone = datetime.datetime.now(datetime.timezone.utc).astimezone().tzinfo
report_title, report_date, report_time, report_timezone

## Data

In [ ]:
local_fig = openbb.stocks.candle(ticker, ticker_data, external_axes=True)
plot_chart = local_fig.to_html()

# Prepare date for forecast
ticker_data = ticker_data.reset_index()
ticker_data.columns = [x.lower() for x in ticker_data.columns]

In [ ]:
corr_chart = get_html(openbb.forecast.corr_chart, None, ticker_data)

In [ ]:
season_chart = get_html(openbb.forecast.season_chart, "Seasonality Data", ticker_data)

In [ ]:
expo_chart = get_html(openbb.forecast.expo_chart, None, ticker_data, legend=True)

In [ ]:
rnn_chart = get_html(openbb.forecast.rnn_chart, None, ticker_data, legend=True)

## Render the report template to a file

In [ ]:
body = ""


body += widgets.header(
    author,
    report_date,
    report_time,
    report_timezone,
    f"<b>FORECAST REPORT ON:</b> {ticker.upper()}",
    plotly_js=True,
)

body += widgets.tablinks(["SUMMARY"])

htmlcode = widgets.h(3, "Summary")
htmlcode += widgets.row([widgets.h(3, "Understand the data")])
htmlcode += widgets.row([ticker_data.head(10).to_html()])
htmlcode += widgets.row([plot_chart])
htmlcode += widgets.row([widgets.h(3, "Analyzing the data")])
htmlcode += widgets.row([corr_chart])
htmlcode += widgets.row([season_chart])
htmlcode += widgets.row([widgets.h(3, "Creating predictions")])
htmlcode += widgets.row([expo_chart])
htmlcode += widgets.row([rnn_chart])

body += widgets.add_tab("SUMMARY", htmlcode)
body += widgets.tab_clickable_and_save_evt()
report = widgets.html_report(title=report_name, stylesheet=stylesheet, body=body)

# to save the results
with open(report_name + ".html", "w", encoding="utf-8") as fh:
    fh.write(report)